# Задание 1.1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [ ]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [ ]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [ ]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [ ]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [ ]:
def foldl2(f, x0, lst):
    return foldr(..., ..., lst)(...)

In [ ]:
def foldr2(f, x0, lst):
    return foldl(..., ..., lst)(...)

# Задание 1.2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [ ]:
def check_inv(a: str, b: str) -> bool:
    for begin in range(len(a)-len(b)+1):
        if sorted(a[begin:begin+len(b)]) == sorted(b):
            return True
    return False
check_inv('abcrotm', 'tro')
#сложность О(len(a)*len(b))

True

# Задание 1.3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [ ]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self, lst=[]):
        if self.left == self.right == None:
            lst.append(self.value)
        if self.left != None:
            iter_tree(self.left, lst)
        if self.right != None:
            iter_tree(self.right, lst)
        return iter(lst)
    
    def __repr__(self):
        if self.right is not None:
            text = '<{}({value!r}, {left!r}, {right!r})>'
        elif self.left is not None:
            text = '<{}({value!r}, {left!r})>'
        else:
            text = '<{}({value!r})>'
        return text.format(type(self).__name__, **vars(self))
    
    def __str__(self):
        return list(self.__iter__())
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

# Задание 1.4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [ ]:
def calc(expr):
    pass

calc('2 * (15 - 3 * 4) - 2') == 4

In [ ]:
import re
from typing import Any, Union

def _is_float(element: str) -> bool:
    try:
        float(element)
        return True
    except ValueError:
        return False

def _get_index(expr: list, value: Any, default: Any):
    try:
        return expr.index(value)
    except ValueError:
        return default

def _split_expr(expr: Union[str, list]):
    if type(expr) == list:
        return expr
    expr = expr.replace(' ', '')
    expr_list = re.split('(\+|\-|\*|\/|\(|\))', expr)
    expr_list = list(filter(None, expr_list))
    minus_to_remove = []
    for i, val in enumerate(expr_list):
        if _is_float(val):
            expr_list[i] = float(val)
        if val == '-' and (i == 0 or not _is_float(expr_list[i-1])):
            expr_list[i+1] = '-'+expr_list[i+1]
            minus_to_remove.append(i)
    for index in sorted(minus_to_remove, reverse=True):
        del expr_list[index]
    return expr_list

# Поиск индекса с закрывающей скобкой
def _get_next_bracket_index(expr: list, index: int):
    opened_brackets = 1
    for i, val in enumerate(expr):
        if val == '(':
            opened_brackets+=1
        if val == ')':
            opened_brackets-=1
        if opened_brackets == 0:
            return i+index+1
        
def calc(expr: str, ndigits: int = 3):
    expr_list = _split_expr(expr)
    while '(' in expr_list:
        bracket_begin = expr_list.index('(')
        bracket_end = _get_next_bracket_index(expr_list[bracket_begin+1:], bracket_begin)
        
        expr_list = expr_list[:bracket_begin] + [calc(expr_list[bracket_begin+1:bracket_end])]+ \
                    expr_list[bracket_end+1:]
    while '*' in expr_list or '/' in expr_list:
        sign_index = min(_get_index(expr_list, '*', len(expr_list)+1), 
                         _get_index(expr_list, '/', len(expr_list)+1))
        if expr_list[sign_index] == '*':
            expr_list = expr_list[:sign_index-1]+[expr_list[sign_index-1]*expr_list[sign_index+1]]+ \
                        expr_list[sign_index+2:]
        elif expr_list[sign_index] == '/':
            expr_list = expr_list[:sign_index-1]+[expr_list[sign_index-1]/expr_list[sign_index+1]]+ \
                        expr_list[sign_index+2:]
    while '+' in expr_list or '-' in expr_list:
        sign_index = min(_get_index(expr_list, '+', len(expr_list)+1), 
                         _get_index(expr_list, '-', len(expr_list)+1))
        
        if expr_list[sign_index] == '+':
            expr_list = expr_list[:sign_index-1]+[expr_list[sign_index-1]+expr_list[sign_index+1]]+ \
                        expr_list[sign_index+2:]
        elif expr_list[sign_index] == '-':
            expr_list = expr_list[:sign_index-1]+[expr_list[sign_index-1]-expr_list[sign_index+1]]+ \
                        expr_list[sign_index+2:]     
    return round(expr_list[0], ndigits)

calc('2 * (15 - 3 * 4) - 2') 